In [0]:
# Importing Libraries 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Meshing for Linear Elastic Beam uncer Plane Stress condition 
# making a list of all nodes in mesh as per ABAQUS 
# Element Connectivity 
#               BEAM under Plane Stress with Dimension 
#    x ------------------------------------------------- x 
#    | (0,6)                                             | (48, 6) 
#    |                                                   |
#    |                                                   |
#    x ------------------------------------------------- x 
#      (0,-6)                                             (48, -6)

x = np.array([0,48,48,0])
y = np.array([-6,-6,6,6])

# Get Max x-length and y-length 
x_length = np.max(x) - np.min(x)
y_length = np.max(y) - np.min(y)

# get mesh points in geometry 
x_linspace = np.linspace(np.min(x), np.max(x), x_length+1, endpoint=True)
y_linspace = np.linspace(np.min(y), np.max(y),y_length+1, endpoint=True)

# loop for making list of points 
x_mesh = []
y_mesh = []
for i in range(len(y_linspace)):
  for j in range(len(x_linspace)):
    x_mesh.append(x_linspace[j])
    y_mesh.append(y_linspace[i])

mesh_file = open('Mesh_data.csv',mode='w')
for i in range(len(x_mesh)):
  mesh_file.write(np.str(i+1))
  mesh_file.write(',')
  mesh_file.write(np.str(x_mesh[i]))
  mesh_file.write(',')
  mesh_file.write(np.str(y_mesh[i]))
  # mesh_file.write(',')
  mesh_file.write('\n')
mesh_file.close()

In [0]:
# Element Connectivity 
Element_connectivity = open('Ele_conn.csv',mode='w')
# Element_connectivity.write('Element_no, node1, node2, node3, node4')
# Element_connectivity.write('\n')
node_1 = []
node_2 = []
node_3 = []
node_4 = []
for j in range(y_length): 
  for i in range(x_length):
    node_1.append((j*(x_length+1)) + (i+1))
    node_2.append((j*(x_length+1)) + (i+2))
    node_4.append(((j+1)*(x_length+1))+(i+1))
    node_3.append(((j+1)*(x_length+1))+(i+2))
for i in range(len(node_1)):
  Element_connectivity.write(np.str(i+1))
  Element_connectivity.write(',')
  Element_connectivity.write(np.str(node_1[i]))
  Element_connectivity.write(',')
  Element_connectivity.write(np.str(node_2[i]))
  Element_connectivity.write(',')
  Element_connectivity.write(np.str(node_3[i]))
  Element_connectivity.write(',')
  Element_connectivity.write(np.str(node_4[i]))
  # Element_connectivity.write(',')
  Element_connectivity.write('\n')

Element_connectivity.close()

In [2]:
# Parabolic load conversion into Concentrated load 
def Parablolic_load(y):
  P = 1000 
  D = 12 
  I = D**3/12 
  tau_xy = (D**2 / 4  - y**2)
  tau_xy = (-P/(2*I)) * tau_xy
  return tau_xy

# Point @ y = -+ 6 (Node -- 49, 637)
load_6 = Parablolic_load(6)

# Point @ y = -+ 5 (Node -- 98, 588)
load_5 = Parablolic_load(5)

# Point @ y = -+ 4 (Node -- 147, 539)
load_4 = Parablolic_load(4)

# Point @ y = -+ 3 (Node -- 196, 490)
load_3 = Parablolic_load(3)

# Point @ y = -+ 2 (Node -- 245, 441)
load_2 = Parablolic_load(2)

# Point @ y = -+ 1 (Node -- 294, 392)
load_1 = Parablolic_load(1)

# Point @ y = -+ 0 (Node -- 343)
load_0 = Parablolic_load(0)

print(load_6)
print(load_5)
print(load_4)
print(load_3)
print(load_2)
print(load_1)
print(load_0)

-0.0
-38.19444444444444
-69.44444444444444
-93.75
-111.11111111111111
-121.52777777777779
-125.0


In [9]:
# Set_1: (48, +-6)
Load_6 = Parablolic_load(6)
print('Load_6', Load_6)

# Set_2: (48, +-5.07692289)
Load_5 = Parablolic_load(5.07692289)
print('Load_5', Load_5)

# Set_3: (48,-+ 4.15384626 )
Load_4 = Parablolic_load(4.15384626)
print('Load_4', Load_4)
 
# Set_4: (48,-+ 3.23076916 )
Load_3 = Parablolic_load(3.23076916)
print('Load_3', Load_3)

# Set_5: (48,-+ 2.30769229 )
Load_2 = Parablolic_load(2.30769229)
print('Load_2', Load_2)

# Set_6: (48,-+ 1.38461542 )
Load_1 = Parablolic_load(1.38461542)
print('Load_1', Load_1)

# Set_6: (48,-+ 0.461538464 )
Load_0 = Parablolic_load(0.461538464)
print('Load_0', Load_0)


Load_6 -0.0
Load_5 -35.50296517011823
Load_4 -65.08875433431949
Load_3 -88.75739803747533
Load_2 -106.50887602317555
Load_1 -118.34319492603551
Load_0 -124.26035502169626


In [0]:
# Input File Generation for Beam Under Parabolic Loading (Plane Stress)
Input_file = open('Beam_Parabolic.inp', mode='w')

# Heading 
Input_file.write('*HEADING')
Input_file.write('\n')
Input_file.write('Plane Stress Linear Static analysis of Beam under Parabolic Loading ')
Input_file.write('\n')

# Preprint 
Input_file.write('*PREPRINT, ECHO=YES, HISTORY=NO, MODEL=NO')
Input_file.write('\n')

# Node List 
Input_file.write('*NODE')
Input_file.write('\n')
with open('Mesh_data.csv') as mesh_data:
  for line in mesh_data:
    Input_file.write(line)
# User Element Type 
Input_file.write('*USER ELEMENT, NODES=4, TYPE=U1, COORDINATES=2, PROPERTIES=2, VARIABLES=8')
Input_file.write('\n')
Input_file.write('1, 2')
Input_file.write('\n')

# Element connectivity 
Input_file.write('*ELEMENT, TYPE=U1, ELSET=EALL')
Input_file.write('\n')
with open('Ele_conn.csv') as ele_conn:
  for line in ele_conn:
    Input_file.write(line)

# UEL Property 
Input_file.write('*UEL PROPERTY, ELSET=EALL')
Input_file.write('\n')
Input_file.write('3e+07, 0.3')
Input_file.write('\n')

# Node Sets 
# Boundary 
Input_file.write('*Nset, nset=Set_boundary')
Input_file.write('\n')
Input_file.write('1, 589, 49')
Input_file.write('\n')
# Parabolic -- concentrated load 
Input_file.write('*Nset, nset=Set_1')
Input_file.write('\n')
Input_file.write('49, 637')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_2')
Input_file.write('\n')
Input_file.write('98, 588')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_3')
Input_file.write('\n')
Input_file.write('147, 539')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_4')
Input_file.write('\n')
Input_file.write('196, 490')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_5')
Input_file.write('\n')
Input_file.write('245, 441')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_6')
Input_file.write('\n')
Input_file.write('294, 392')
Input_file.write('\n')

Input_file.write('*Nset, nset=Set_7')
Input_file.write('\n')
Input_file.write('343')
Input_file.write('\n')

# Boundary conditions 
Input_file.write('*BOUNDARY')
Input_file.write('\n')
Input_file.write('Set_boundary, ENCASTRE')
Input_file.write('\n')

# Step 
Input_file.write('*STEP')
Input_file.write('\n')
Input_file.write('*STATIC')
Input_file.write('\n')
Input_file.write('1.,1.,1e-05,1.')
Input_file.write('\n')

# Concentrated Force 
Input_file.write('*CLOAD')
Input_file.write('\n')
Input_file.write('Set_1, 2, 0.0')
Input_file.write('\n')
Input_file.write('Set_2, 2, -38.194')
Input_file.write('\n')
Input_file.write('Set_3, 2, -69.44')
Input_file.write('\n')
Input_file.write('Set_4, 2, -93.75')
Input_file.write('\n')
Input_file.write('Set_5, 2, -111.11')
Input_file.write('\n')
Input_file.write('Set_6, 2, -121.52')
Input_file.write('\n')
Input_file.write('Set_7, 2, -125.0')
Input_file.write('\n')

# Output Requests 
Input_file.write('*OUTPUT,FIELD')
Input_file.write('\n')
Input_file.write('*ENERGY FILE')
Input_file.write('\n')
Input_file.write('*OUTPUT,HISTORY,FREQUENCY=1')
Input_file.write('\n')
Input_file.write('*ENERGY OUTPUT,VARIABLE=PRESELECT')
Input_file.write('\n')
Input_file.write('*NODE FILE')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*OUTPUT,FIELD')
Input_file.write('\n')
Input_file.write('*NODE OUTPUT')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*NODE PRINT')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*EL PRINT')
Input_file.write('\n')
Input_file.write('SDV')
Input_file.write('\n')
Input_file.write('*END STEP')

Input_file.close()